In [1]:
import pygame
import os
pygame.font.init()

WIN_WIDTH = 1200
WIN_HEIGHT = 650

win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
MOTOR_IMG = pygame.image.load(os.path.join('motor', 'motor.png')).convert_alpha()
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join('motor', 'back.png')).convert_alpha())
STONE_IMG = pygame.image.load(os.path.join('motor', 'stone.png')).convert_alpha()

STAT_FONT = pygame.font.SysFont("comicsans", 50)

fps = 30
motor_origin_y = 320
Jump = False

# Difficulty
pace = 40
jump_height = 200
gravity = 35

class Motor:
    IMG = MOTOR_IMG
    going_up = True
    jump_height = jump_height
    origin_y = motor_origin_y

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.going_up = self.going_up

    def jump(self):
        global Jump

        if self.y == self.origin_y:
            self.going_up = True

        if self.going_up:
            self.y -= gravity
            if self.y <= (self.origin_y - self.jump_height):
                self.going_up = False
        else:
            self.y += gravity
            if self.y >= self.origin_y:
                Jump = False

    def draw(self, win):
        win.blit(self.IMG, (self.x, self.y))

    def get_mask(self):
        return pygame.mask.from_surface(self.IMG)

class Stone:
    VEL = pace
    IMG = STONE_IMG
    appearing_y_level = motor_origin_y + 150
    appearing_x_level = WIN_WIDTH

    def __init__(self):
        self.passed = False

    def move(self):
        self.appearing_x_level -= self.VEL

    def draw(self, win):
        win.blit(self.IMG, (self.appearing_x_level, self.appearing_y_level))

    def collide(self, motor):
        motor_mask = motor.get_mask()
        stone_mask = pygame.mask.from_surface(self.IMG)

        # calculate offset between masks
        offset = (self.appearing_x_level - motor.x, self.appearing_y_level - motor.y)

        collided = motor_mask.overlap(stone_mask, offset)

        if collided:
            return True
        return False

class Background:
    VEL = pace
    WIDTH = BG_IMG.get_width()
    IMG = BG_IMG

    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH

    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH

        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))


def draw_window(win, background, motor, stones, score):
    background.draw(win)
    motor.draw(win)
    for stone in stones:
        stone.draw(win)

    text = STAT_FONT.render("Score: " + str(score), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))

    pygame.display.update()

def main():
    global Jump
    stones = [Stone()]
    clock = pygame.time.Clock()
    motor = Motor(100, motor_origin_y)
    background = Background(-185)
    background.draw(win)
    win.blit(BG_IMG, (0, 0))
    score = 0

    run = True
    while run:
        clock.tick(fps)

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    Jump = True
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()

        background.move()

        if Jump:
            motor.jump()

        rem = []
        for stone in stones:
            if stone.collide(motor):
                run = False
                break

            if not stone.passed and stone.appearing_x_level < motor.x:
                stone.passed = True
                score += 1

            stone.move()

            if stone.appearing_x_level + stone.IMG.get_width() < 0:
                rem.append(stone)
                stones.append(Stone())

        for r in rem:
            stones.remove(r)

        win.fill((0,0,0))
        draw_window(win, background, motor, stones, score)


while True:
    main()
    Jump = False


pygame 2.0.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit